In [1]:
import os
import glob
import oommfodt
import numpy as np
import traitlets
import ipywidgets
import discretisedfield as df
import matplotlib.pyplot as plt
import IPython.display
from IPython.display import display, clear_output
%matplotlib inline

In [2]:
class DataAnalysisODT:
    def __init__(self, name):
        self.dirname = os.path.join(name, "")
        
        odtfile = max(glob.iglob("{}*.odt".format(self.dirname)),
                      key=os.path.getctime)
        self.dt = oommfodt.OOMMFodt(odtfile).df
        self.time = self.dt["t"].as_matrix()/1e-9
        
        self.slider = ipywidgets.FloatRangeSlider(value=[self.time[0], self.time[-1]],
                                                  min=self.time[0], 
                                                  max=self.time[-1],
                                                  step=self.time[1]-self.time[0],
                                                  description="t (ns):",
                                                  readout=True,
                                                  layout=ipywidgets.Layout(width="100%"))

        self.select = ipywidgets.Select(options=list(self.dt.columns.values),
                                        description="Output:")
        
        self.slider.observe(self.update_output_slider)
        self.select.observe(self.update_output_slider)
        self.out = ipywidgets.Output(layout=ipywidgets.Layout(width="300%"))
    
        self.update_output_slider(None)
        
    def _ipython_display_(self):
        box1 = ipywidgets.VBox([self.slider, self.out])
        box2 = ipywidgets.HBox([self.select, box1])
        display(box2)
            
    def update_output_slider(self, value):
        #IPython.display.clear_output(wait=True)
        self.out.clear_output(wait=True)
        plt.plot(self.time, self.dt[self.select.value])
        plt.xlabel("t (ns)")
        plt.ylabel(self.select.value)
        plt.xlim([self.slider.value[0], self.slider.value[1]])
        plt.grid()
        with self.out:
            display(plt.gcf())
        plt.close()

In [3]:
DataAnalysisODT(name="stdprob4")

In [33]:
class DataAnalysisOMF:
    def __init__(self, name):
        self.dirname = os.path.join(name, "")
        
        odtfile = max(glob.iglob("{}*.odt".format(self.dirname)),
                      key=os.path.getctime)
        self.omffiles = sorted(glob.iglob("{}*.omf".format(self.dirname)),
                               key=os.path.getctime)
        last_omf_file = max(glob.iglob("{}*.omf".format(self.dirname)),
                            key=os.path.getctime)
        self.last_field = df.read_oommf_file(last_omf_file)
        
        self.dt = oommfodt.OOMMFodt(odtfile).df
        self.stage = self.dt["stage"].as_matrix()
        
        self.slider = ipywidgets.IntSlider(value=self.stage[0],
                                           min=self.stage[0], 
                                           max=self.stage[-1],
                                           step=self.stage[1]-self.stage[0],
                                           description="Stage:",
                                           readout=True,
                                           layout=ipywidgets.Layout(width="80%"))
        
        self.play = ipywidgets.Play(value=self.stage[0],
                                    min=self.stage[0], 
                                    max=self.stage[-1],
                                    step=self.stage[1]-self.stage[0])

        self.options = {"x": 0, "y": 1, "z": 2}
        self.select = ipywidgets.RadioButtons(options=list(self.options.keys()),
                                              description="Slice:")
        
        self.slice_slider = ipywidgets.FloatSlider(value=self.last_field.mesh.pmin[0]/1e-9,
                                                   min=self.last_field.mesh.pmin[0]/1e-9, 
                                                   max=self.last_field.mesh.pmax[0]/1e-9,
                                                   step=self.last_field.mesh.cell[0]/1e-9,
                                                   description="x:",
                                                   readout=True,
                                                   layout=ipywidgets.Layout(width="80%"))
        
        ipywidgets.jslink((self.play, "value"), (self.slider, "value"))

        self.slider.observe(self.update_plot)
        self.select.observe(self.update_slider)
        self.out = ipywidgets.Output(layout=ipywidgets.Layout(width="300%"))
    
        self.update_plot(None)
        
    def _ipython_display_(self):
        box1 = ipywidgets.VBox([self.slider, self.slice_slider, self.out])
        box2 = ipywidgets.HBox([self.select, box1])
        display(box2)
        display(self.play)
            
    def update_plot(self, value):
        self.out.clear_output(wait=True)
        omffile = self.find_omf_file()
        field = df.read_oommf_file(omffile)
        field.plot_slice(self.select.value, 0, xsize=10)
        with self.out:
            display(plt.gcf())
        plt.close()
        
    def update_slider(self, value):
        i = self.options[self.select.value]
        self.slice_slider.value = self.last_field.mesh.pmin[i]/1e-9,
        self.slice_slider.min = self.last_field.mesh.pmin[i]/1e-9, 
        self.slice_slider.max = self.last_field.mesh.pmax[i]/1e-9,
        self.slice_slider.step = self.last_field.mesh.cell[i]/1e-9,
        self.slice_slider.description = self.select.value
        
        self.update_plot()
            
    def find_omf_file(self):
        for f in self.omffiles:
            if str(self.slider.value) in f[0:-7]:
                return f

In [34]:
DataAnalysisOMF(name="stdprob4")

TraitError: The 'value' trait of a FloatSlider instance must be a float, but a value of (0.0,) <class 'tuple'> was specified.

In [18]:
options = {"x": 0, "y": 1, "z": 2}

In [20]:
list(options.keys())

['x', 'z', 'y']